In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Load your dataset
df = pd.read_csv('/content/final_dataset - Sheet1.csv')

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize the comments
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['Comment'])

# Convert comments to sequences
train_sequences = tokenizer.texts_to_sequences(train_df['Comment'])
test_sequences = tokenizer.texts_to_sequences(test_df['Comment'])

# Pad sequences
max_sequence_length = max(len(seq) for seq in train_sequences)
X_train = pad_sequences(train_sequences, maxlen=max_sequence_length)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Prepare target data
y_train = np.array(train_df['Stance'])
y_test = np.array(test_df['Stance'])

# Prepare target-specific information
target_info_train = np.array(train_df['Target'])
target_info_test = np.array(test_df['Target'])

# Build TAN model
embedding_dim = 100

# Input layers
input_comment = Input(shape=(max_sequence_length,), name='input_comment')
input_target = Input(shape=(1,), name='input_target')

# Embedding layer for comments
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(input_comment)

# Bidirectional LSTM layer
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)

# Attention layer
# attention_layer = Attention()([lstm_layer, tf.expand_dims(input_target, axis=-1)])
attention_layer = Attention()([lstm_layer, tf.tile(tf.expand_dims(input_target, axis=-1), multiples=[1, 1, 128])])


# Concatenate attention output with target information
merged_layer = Concatenate()([lstm_layer, attention_layer])

# GlobalMaxPooling1D layer
global_max_pooling_layer = GlobalMaxPooling1D()(merged_layer)

# Dense layers
dense_layer = Dense(32, activation='relu')(global_max_pooling_layer)
output_layer = Dense(1, activation='sigmoid', name='output')(dense_layer)

# Build and compile the model
model = Model(inputs=[input_comment, input_target], outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#
# Train the model
model.fit([X_train, target_info_train], y_train, epochs=10, batch_size=32, validation_split=0.2)


# Evaluate the model on the test set
y_pred = model.predict([X_test, target_info_test])
y_pred_binary = (y_pred > 0.5).astype(int)

# Overall metrics
test_loss, test_accuracy = model.evaluate([X_test, target_info_test], y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')

# Calculate metrics for each target class
unique_targets = df['Target'].unique()

for target_class in unique_targets:
    indices = test_df[test_df['Target'] == target_class].index
    indices = indices[indices < len(y_test)]  # Ensure indices are within bounds

    y_test_target = y_test[indices]
    y_pred_target = y_pred_binary[indices]

    accuracy = accuracy_score(y_test_target, y_pred_target)
    precision = precision_score(y_test_target, y_pred_target)
    recall = recall_score(y_test_target, y_pred_target)
    f1 = f1_score(y_test_target, y_pred_target)
    cm = confusion_matrix(y_test_target, y_pred_target)

    print(f'Target {target_class} Metrics:')
    print(f'  Accuracy: {accuracy * 100:.2f}%')
    print(f'  Precision: {precision:.4f}')
    print(f'  Recall: {recall:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f' Confusion Matrix:\n{cm}')
    print('\n')

Epoch 1/10
110/110 [==============================] - 92s 776ms/step - loss: 0.6181 - accuracy: 0.6890 - val_loss: 0.5670 - val_accuracy: 0.6690
Epoch 2/10
110/110 [==============================] - 84s 758ms/step - loss: 0.3111 - accuracy: 0.8780 - val_loss: 0.3092 - val_accuracy: 0.8832
Epoch 3/10
110/110 [==============================] - 89s 811ms/step - loss: 0.1028 - accuracy: 0.9662 - val_loss: 0.3164 - val_accuracy: 0.8900
Epoch 4/10
110/110 [==============================] - 83s 759ms/step - loss: 0.0416 - accuracy: 0.9885 - val_loss: 0.3959 - val_accuracy: 0.8935
Epoch 5/10
110/110 [==============================] - 81s 740ms/step - loss: 0.0259 - accuracy: 0.9928 - val_loss: 0.4462 - val_accuracy: 0.8969
Epoch 6/10
110/110 [==============================] - 80s 725ms/step - loss: 0.0144 - accuracy: 0.9966 - val_loss: 0.4915 - val_accuracy: 0.8981
Epoch 7/10
110/110 [==============================] - 83s 754ms/step - loss: 0.0101 - accuracy: 0.9977 - val_loss: 0.5251 - val_ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and bei